In [2]:
import pandas as pd

In [5]:
JJAS_trips = pd.read_csv('sf_bikesharetrips_2017_junjulaugsept.csv')
OND_trips = pd.read_csv('sf_bikesharetrips_2017_octnovdec.csv')

In [ ]:
sf_bikshare_trips = JJAS_trips.append